# AES example to encrypt and decrypt a message 

## Encryption part

In [36]:
#!/usr/bin/python

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

with open("aes.key", "wb") as file_out:
    key = get_random_bytes(16)
    file_out.write(key)
data = "plaintext for AES"
cipher = AES.new(key, AES.MODE_EAX)
cipher_text, tag = cipher.encrypt_and_digest(data.encode())
with open("encrypted.bin", "wb") as file_out:
    [file_out.write(x) for x in (cipher.nonce, tag, cipher_text)]
print("Data is encrypted and stored in a file")




Data is encrypted and stored in a file


AES cypher object is created by passing the key. Cypher mode EAX is used in the above code. This object is used to encrypt the data. Nonce, tag and ciphertext are stored and transmitted to the decryption end.

## Decryption part

In [37]:


with open("aes.key", "rb") as file_in:
    key = file_in.read(16)
with open("encrypted.bin", "rb") as file_in:
    nonce, tag, cipher_text = [file_in.read(x) for x in (16, 16, -1)]

# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(cipher_text, tag)
print("Decrypted data is : \"{}\"".format(data.decode()))

Decrypted data is : "plaintext for AES"


Decryption part of the AES uses same 16-byte symmetric key generated at the encryption end. Ideally this key has to be tranferred using a secure channel to the other end. Received encrypted binary file read to get nonce, tag and the ciphertext itelf. AES cipher object is created using the same key and nonce value.  Finally decryption is performed using decrypt_and_verify method by providing ceipher text and tag value. Tag value is provided to perform verfication which checks for any modification in the cipher text.  

# RSA example to encrypt and decrypt a message 

## Encryption part

In [31]:
#!/usr/bin/python

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP


message = "plaintext for RSA"

key = RSA.generate(2048)
public = key.publickey()

cipher = PKCS1_OAEP.new(public)
cipher_text = cipher.encrypt(message.encode())
print("Data is encrypted")




Data is encrypted


RSA key of 2048 bits is created using the generate method of RSA package. Public key is exported from this generated key and made public. The entire key object should be kept secret. A cipher object is created using the public key and encryption is performed on message using this object.

## Decryption part

In [32]:

cipher = PKCS1_OAEP.new(key)
message = cipher.decrypt(cipher_text)
print("Decrypted data is : \"{}\"".format(message.decode()))

Decrypted data is : "plaintext for RSA"


Decryption operation is performed similar to the encryption side, but private part of the key pair is used instead of exposed part. Available cipher text is given as input to the decrypt method which decrypts and gives back the message. 

# ECDSA example to create and verify a digital signatur

## ECC key generation

In [33]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS

key = ECC.generate(curve='P-256')

with open('ecc.pub', 'wt') as f:
    f.write(key.public_key().export_key(format='PEM'))
with open('ecc.pem', 'wt') as f:
    f.write(key.export_key(format='PEM'))




Key is generated on the Secp256k1 elliptical curve using ECC.generate method, and both public and private keys are exported. 

## creation digital signature

In [34]:

message = b'ECDSA message for signature'
key = ECC.import_key(open('ecc.pem').read())
h = SHA256.new(message)
signer = DSS.new(key, 'fips-186-3')
signature = signer.sign(h)



Messages which needs to be signed is hashed using SHA256 algorithm and then a signer object is created using DSS package by providing private key. Hashed message is then signed by the owner.

## verifying the  digital signature

In [35]:

h = SHA256.new(message)
key = ECC.import_key(open('ecc.pub').read())
verifier = DSS.new(key, 'fips-186-3')
try:
    verifier.verify(h, signature)
    print("The message is authentic.")
except ValueError:
    print("The message is not authentic.")

The message is authentic.


Signature verification in the above code is similar to that of signing. Received message is hashed initially since it was performed at the sender side as well. Distributed public key is imported and used to create a new DSS object for verification. Hashed message and received signature is used for the verification. Verify function would throw a ValueError if the message or signature was tampered. 